# Testing cocoa.geo.GeoManager

## Importing data
If executed locally

In [ ]:
import sys
sys.path.insert(1, '..')

If executed on a server (e.g. [Google Colab](https://colab.research.google.com/) ), you should install cocoa via `pip3`

In [ ]:
!pip3 install -q git+https://github.com/tjbtjbtjb/CoCoA.git

In [ ]:
from cocoa.geo import GeoManager as gm

## Getting some help

In [ ]:
help(gm)

## Usage

In [ ]:
g=gm() # Create the instance of GeoManager 

### Suppported options

In [ ]:
g.get_list_db()

In [ ]:
g.get_list_output()

In [ ]:
g.get_list_standard()

In [ ]:
g.get_standard()

### Changing the default output standard

In [ ]:
g.set_standard('name')

In [ ]:
g2=gm('iso3')
g2.get_standard()

### Converting countries to standardized names

In [ ]:
g.to_standard('england') # single country

In [ ]:
g.to_standard(['esp','it']) # list of countries

In [ ]:
from cocoa.error import * 
try:
    z=g.to_standard('European Union') # the name does not exist as a country
except CocoaError:
    print('The input name is unknown, try to interpret as a region')
    z=g.to_standard('European Union',interpret_region=True)
print(z)

In [ ]:
g.to_standard(['usa','South America','russia'],interpret_region=True) # mix between region and countries

In [ ]:
g.to_standard('french') # difficult interpretation, getting first item, a warning appears

### Conversion for specific databases

In [ ]:
try:
    z=g.to_standard('Congo (Kinshasa)')
except CocoaError:
    print('Try to interpret within the JHU database country name usage')
    z=g.to_standard('Congo (Kinshasa)',db='jhu')
z

### Other outputs

In [ ]:
g.to_standard(['fr','spain','england'],output='dict')

In [ ]:
g.to_standard(['fr','spain','england'],output='pandas')

## Management of errors

As far as possible, errors are managed within the `cocoa.error` framework. `CocoaError` should be raised.

In [ ]:
g.set_standard('oups') # the standard is not in the standard list 

In [ ]:
g3=gm('hi!') # idem

In [ ]:
g.to_standard('Congo (Kinshasa)',db='another base') # unknown base

In [ ]:
g.to_standard('aus',output='another output') # bad output type

In [ ]:
g.to_standard('Europe',interpret_region=1) # bad type (boolean required) for interpret_region option

In [ ]:
g.to_standard('elsewhere') # unknown country

In [ ]:
g.to_standard('European Union',output='dict',interpret_region=True) # cannont create dict or pandas output with interpret_region=True